# **Como selecionar as melhores features para um modelo de ML?**

3 principais conceitos:
- Embedded methods
- Filter methods
- Wrapper methods

# **Embedded methods**
métodos de seleção que estão dentro do modelo, ex: LASSO (Least Absolute Shrinkage and Selection) -> Add uma penalização, encorajando os coeficientes de menos importância para zero. Ou seja, usa algoritmos de penalização para penalizar os coeficientes de menos importância, ou valorizam os coeficientes de features consideradas mais importantes.
Cuidados: - Calibração dos hyperparametros

# **Filter methods**
caracteristicas das features e técnicas estatísticas para a seleção das features. Medindo a relação entre a possível feature para entrar no modelo e a variável alvo. O subset das melhores rankeadas são usadas para o modelo.

- utiliza testes estátisticos e medidas para inferir o relacionamento entre os inputs e a variável alvo -> antes de treinar o modelo, seleciona o melhor subset das features
- Primeiro começar com uma matriz de correlação, para tirar features que possuam alto nível de correlação, pois assim voce escolhe uma que tiver mais sentido, simplificando o modelo e ja ira englobar os efeitos das excluidas

### **Seleção de features categóricas**:
2 teste: teste de qui-quadrado e Anova-F Statistic

#### Chi-quadrado - (Variáveis Categóricas)
- determina a extensão do relacionamento ou a dependencia entre 2 variáveis categóricas (input x target)
- Chi-Quadrado é um teste para saber se variáveis são dependentes uma da outra
- Valores altos de Chi-Quadrado indicam variáveis mais dependentes e melhores para serem usadas no modelo.

#### Anova-F - (Variáveis Contínuas)
- mede a independencia de 2 variáveis contínuas (maior o valor, maior a dependência)
- ANOVA mede as relações entre grupos de dados inferindo a dependência entre eles a partir de diferenças nas médias.
- ANOVA se baseia em valores de F. Quanto maior o valor de F, maior a diferença entre grupos e, logo, maior impacto uma variável causa na outra, sendo um bom indicativo para selecionar a variável para seu modelo.

#### **Categorical variables**

In [ ]:
# import all the required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import datasets

# carrega o dataset
df = pd.read_csv('https://raw.githubusercontent.com/vinihora/portfolio_data_science/main/dados/Placement_Data_Full_Class.csv')

In [ ]:
df

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,211,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0
211,212,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0
212,213,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0
213,214,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0


In [ ]:
# Descobre o formato de cada uma das variáveis
df.dtypes

sl_no               int64
gender             object
ssc_p             float64
ssc_b              object
hsc_p             float64
hsc_b              object
hsc_s              object
degree_p          float64
degree_t           object
workex             object
etest_p           float64
specialisation     object
mba_p             float64
status             object
salary            float64
dtype: object

In [ ]:
# Cria um df para o target
y = df['status']

features = []
for i in df.columns:
  if i not in ['status', 'ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p', 'salary', 'sl_no']:
    features.append(i)

# Cria um df para os inputs
X = df[features]

In [ ]:
X

,gender,ssc_b,hsc_b,hsc_s,degree_t,workex,specialisation
0,M,Others,Others,Commerce,Sci&Tech,No,Mkt&HR
1,M,Central,Others,Science,Sci&Tech,Yes,Mkt&Fin
2,M,Central,Central,Arts,Comm&Mgmt,No,Mkt&Fin
3,M,Central,Central,Science,Sci&Tech,No,Mkt&HR
4,M,Central,Central,Commerce,Comm&Mgmt,No,Mkt&Fin
...,...,...,...,...,...,...,...
210,M,Others,Others,Commerce,Comm&Mgmt,No,Mkt&Fin
211,M,Others,Others,Science,Sci&Tech,No,Mkt&Fin
212,M,Others,Others,Commerce,Comm&Mgmt,Yes,Mkt&Fin
213,F,Others,Others,Commerce,Comm&Mgmt,No,Mkt&HR


In [ ]:
# Chama o métode de encode de ordinais -> transforma variáveis categóricas em ordinais
oe = OrdinalEncoder()

# treina o método para o seu df
oe.fit(X)

# transform training and test sets and convert to DFs
X_enc = pd.DataFrame(oe.transform(X), columns=X.columns)
y_enc = pd.DataFrame(oe.transform(X), columns=X.columns)

In [ ]:
X_enc.dropna(inplace=True)

In [ ]:
le = LabelEncoder()
# fit the LabelEncoder class on training set
le.fit(y)
# transform training and test target variables and convert to DFs
y = le.transform(y)

In [ ]:
chi_scores = chi2(X_enc,y)

scores = pd.Series(chi_scores[0], index=X.columns)
pvalues = pd.Series(chi_scores[1], index=X.columns)

pd.DataFrame({'Chi2':scores, 'p-Value':pvalues})

,Chi2,p-Value
gender,0.624805,0.429267
ssc_b,0.161359,0.687908
hsc_b,0.024118,0.876585
hsc_s,0.058876,0.808282
degree_t,0.117084,0.732219
workex,10.745484,0.001045
specialisation,7.539357,0.006037


In [ ]:
# Divide o df em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size=0.33, random_state=1, stratify = y)

In [ ]:
# A partir do teste de chi2, podemos observar que as colunas workex e specialisation possuem um nível muito elevado das demais no teste chi-quadrado, inferindo que podem ser as 2 melhores a serem utilizadas. Além disso, foram as únicos a apresentarem um p-value menor que 0.05.
fs = SelectKBest(score_func=chi2, k=2)
# fit on training features and target
fs.fit(X_train, y_train)

SelectKBest(k=2, score_func=<function chi2 at 0x7f7564feba30>)

In [ ]:
X_train_fs = pd.DataFrame(fs.transform(X_train), columns = X_train.columns[fs.get_support()])
X_test_fs = pd.DataFrame(fs.transform(X_test), columns = X_test.columns[fs.get_support()])

In [ ]:
X_train_fs.columns

Index(['workex', 'specialisation'], dtype='object')

Assim como o esperado, as 2 features com maior nível de chi2, foram workex e specialisation. Sendo as 2 selecionadas.

In [ ]:
model = LogisticRegression()
model.fit(X_train_fs, y_train)
yhat = model.predict(X_test_fs)

In [ ]:
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 69.01


#### **Continuous variables**

Utilizaremos o mesmo dataset do método anterior, porém utili

In [ ]:
import pandas as pd
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv('https://raw.githubusercontent.com/vinihora/portfolio_data_science/main/dados/Placement_Data_Full_Class.csv')

In [ ]:
# Cria um df para o target
y = df['status']

features = []
for i in df.columns:
  if i in ['ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p', 'sl_no']:
    features.append(i)

# Cria um df para os inputs
X = df[features]

In [ ]:
X

,sl_no,ssc_p,hsc_p,degree_p,etest_p,mba_p
0,1,67.00,91.00,58.00,55.0,58.80
1,2,79.33,78.33,77.48,86.5,66.28
2,3,65.00,68.00,64.00,75.0,57.80
3,4,56.00,52.00,52.00,66.0,59.43
4,5,85.80,73.60,73.30,96.8,55.50
...,...,...,...,...,...,...
210,211,80.60,82.00,77.60,91.0,74.49
211,212,58.00,60.00,72.00,74.0,53.62
212,213,67.00,67.00,73.00,59.0,69.72
213,214,74.00,66.00,58.00,70.0,60.23


In [ ]:
le = LabelEncoder()
# fit the LabelEncoder class on training set
le.fit(y)
# transform training and test target variables and convert to DFs
y = le.transform(y)

In [ ]:
y

array([1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0])

In [ ]:
# Calculando a estatística com ANOVA
print(f_classif(X, y)[0].round(4))
print(f_classif(X, y)[1].round(4))

[  0.1538 124.8425  67.7452  63.7192   3.5276   1.2678]
[0.6953 0.     0.     0.     0.0617 0.2614]


Ests teste mede quantas variáveis devem ser selecionadas para sem utilizadas no modelo final. O teste de ANOVA-F mostrou que deve se utilizar somente 3 das 6 features, a fim de atingir um melho resultador. (ssc_p, hsc_p, degree_p). Assim, essas 3 apresentaram grande dependência com a variável target.

In [ ]:
X_filtred = X[['ssc_p', 'hsc_p', 'degree_p']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify = y)
X_train_filtred, X_test_filtred, y_train_filtred, y_test_filtred = train_test_split(X_filtred, y, test_size=0.33, random_state=42, stratify = y)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

model_filtred = LogisticRegression()
model_filtred.fit(X_train_filtred, y_train_filtred)

yhat = model.predict(X_test)
yhat_filtred = model_filtred.predict(X_test_filtred)

In [ ]:
accuracy = accuracy_score(y_test, yhat)
accuracy_filtred = accuracy_score(y_test_filtred, yhat_filtred)

print('Accuracy: %.2f' % (accuracy*100))
print('Accuracy Filtred DF: %.2f' % (accuracy_filtred*100))

Accuracy: 74.65
Accuracy Filtred DF: 81.69


Como se pode observar, o DF que utilizou somente as variáveis que mostraram forte dependência com o target atingiu uma acurácia superior ao DF com todas as variáveis disponíveis.

# **Wrapper methods**

Usa ML algorithms como parte da medição e evaluation das features, identificando o melhor subset de features, de acordo com uma métrica de performance específica, ditada pelo cientista. Exemplo: Recursive Feature Elimination (FRE) do scikit-learn